Name: Hamidreza Yaghoubi Araghi

Student ID: 98109786

In this exercise, you should develop a character-level RNN language model.

You are free to choose the architecture, but you must use GRUs and not LSTMs. A linear embedding layer (hidden size 64), a 2-layer GRU (hidden size 128, dropout 0.1), and a linear classifier head is an example architecture.

You should generate some example outputs using beam search.

Some parts of the code has been done for you. You need to implement the parts that raise `NotImplementedError`.

The index zero has been reserved for the padding token/character. By subtracting one from the token indices, the indices will become ASCII indices. (And the padding index will become `-1`.)

The model's classification head should directly predict ASCII characters (256 possibilities). It should not predict any special tokens, such as padding, start or end.

# Bootstrap

# Install

In [ ]:
! pip install -U torch datasets pyperclip icecream numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.

# Download the Data

In [ ]:
!wget https://files.lilf.ir/Black%20Luminary.txt

--2023-05-06 11:37:10--  https://files.lilf.ir/Black%20Luminary.txt
Resolving files.lilf.ir (files.lilf.ir)... 82.102.11.148
Connecting to files.lilf.ir (files.lilf.ir)|82.102.11.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3148450 (3.0M) [text/plain]
Saving to: ‘Black Luminary.txt’

Black Luminary.txt  100%[===================>]   3.00M  1.92MB/s    in 1.6s    

2023-05-06 11:37:13 (1.92 MB/s) - ‘Black Luminary.txt’ saved [3148450/3148450]



In [ ]:
! ls -lh

total 3.1M
-rw-r--r-- 1 root root 3.1M Oct 14  2021 'Black Luminary.txt'
drwxr-xr-x 1 root root 4.0K May  3 13:31  sample_data


In [ ]:
! realpath *.txt

/content/Black Luminary.txt


# User Config

In [ ]:
data_paths = [
    '/content/Black Luminary.txt',
    ]

## imports

In [ ]:
import pyperclip

In [ ]:
from icecream import ic

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device("cpu")
#: We will set device again in the training loop.

In [ ]:
import datasets as D

In [ ]:
import numpy
np = numpy

import statistics

# Utils

In [ ]:
class NumpyPrintOptions:
    def __init__(self, **kwargs):
        self.options = kwargs
        self.original_options = np.get_printoptions()

    def __enter__(self):
        np.set_printoptions(**self.options)

    def __exit__(self, exc_type, exc_value, traceback):
        np.set_printoptions(**self.original_options)

class NoTruncationNumpyPrintOptions(NumpyPrintOptions):
    def __init__(self):
        super().__init__(
            threshold=np.inf,
            linewidth=200,
            suppress=True,
            precision=4
        )

In [ ]:
import jax

def torch_shape_get(input):
    def h_shape_get(x):
        return x.dtype, x.shape

    return jax.tree_map(h_shape_get, input)

In [ ]:
def has_nan(tensor):
    return torch.any(torch.isnan(tensor))

In [ ]:
class ModelEvalMode:
    def __init__(self, model):
        self.model = model

    def __enter__(self):
        self.model.eval()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.model.train()

# Data

In [ ]:
d = D.load_dataset("text",
                         data_files=data_paths, sample_by="paragraph")
d

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-751f099585d07f63/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18423
    })
})

In [ ]:
d = d['train']
d

Dataset({
    features: ['text'],
    num_rows: 18423
})

In [ ]:
d[1000:1010]

{'text': ['Professor Snape threw him backwards, and Harry stumbled, but just managed to keep standing.',
  "'I understand, sir. I apologise if my careless words have offended.'",
  "'Get going then!' The man turned around and marched towards his desk.",
  'Not keen on his company for the moment, Harry hastily made his way towards the hall.',
  "Merlin, did he have to grab my arm like that? If being implicated in murder is child's play to him, then I can indeed do without partaking in his 'problems'…",
  '~BLHD~',
  "Harry paused before entering the Great Hall and forced his countenance as hard as he could into a blank expression. He must not let up; he must not relent for one second. Weakness would not help him here. It might also be prudent to distance himself a bit from other people for a while to limit the damage to their reputation and family; depending on how the whole situation turned out, the political fallout could be immense. With a sense of foreboding, he imagined Daphne's re

In [ ]:
def str_to_np(s, dtype=np.int8):
    s = s.encode('ascii', errors='ignore')
    return np.frombuffer(s, dtype=dtype)

str_to_np('hello')

array([104, 101, 108, 108, 111], dtype=int8)

In [ ]:
def str_to_onehot(s):
    return np.eye(256)[str_to_np(s)]

In [ ]:
dc = d.map(lambda batch: {'input': [str_to_np(t).astype(np.int32) + 1 for t in batch['text']]}, batched=True) #: added one to the char indices to make zero available for the pad token
dc

Map:   0%|          | 0/18423 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input'],
    num_rows: 18423
})

In [ ]:
dc = dc.filter(lambda x: (len(x['input']) > 30 and len(x['text'].split()) > 4), batched=False)
dc

Filter:   0%|          | 0/18423 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input'],
    num_rows: 16371
})

In [ ]:
dc.set_format("torch", columns=["input",])

In [ ]:
torch_shape_get(dc[1000:1010]['input'])

[(torch.int64, torch.Size([1126])),
 (torch.int64, torch.Size([727])),
 (torch.int64, torch.Size([163])),
 (torch.int64, torch.Size([232])),
 (torch.int64, torch.Size([106])),
 (torch.int64, torch.Size([88])),
 (torch.int64, torch.Size([82])),
 (torch.int64, torch.Size([69])),
 (torch.int64, torch.Size([127])),
 (torch.int64, torch.Size([64]))]

In [ ]:
dc = dc.shuffle()

In [ ]:
dcs = dc.train_test_split(test_size=0.2)
dcs

DatasetDict({
    train: Dataset({
        features: ['text', 'input'],
        num_rows: 13096
    })
    test: Dataset({
        features: ['text', 'input'],
        num_rows: 3275
    })
})

# Model

- [GRU --- PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html)

- [torch.nn.utils.rnn.pack_sequence --- PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_sequence.html#torch.nn.utils.rnn.pack_sequence) (not necessarily needed)

- [Embedding --- PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

- [torch.nn.utils.rnn.pad_sequence --- PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html)


In [ ]:
from torch.autograd import Variable

class Model(nn.Module):

    def __init__(self, input_size=257, hidden_size=128, output_size=256, n_layers=2):
      super(Model, self).__init__()
      self.input_size = input_size
      self.hidden_size = hidden_size
      self.output_size = output_size
      self.n_layers = n_layers
      self.encoder = nn.Embedding(input_size, 128)
      self.gru = nn.GRU(128, 512, n_layers, batch_first=True)
      self.decoder = nn.Linear(512, output_size)


    def forward(self, x, hidden=None):
        #: x: list of tensors shaped (seq_length, 256)
        #: The seq_length will not necessarily be equal for all list items!
        ##
        x = self.encoder(x.view(-1, 1))
        output, hidden = self.gru(x, hidden)
        output = self.decoder(output)
        return output[:, 0, :], hidden

    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, batch_size, 512))

In [ ]:
import torch.nn.functional as F

def loss_fn(y, y_hat):
    #: y: list of tensors shaped (seq_length, classes_n+1)
    #: y_hat: tensor shaped (B, T, classes_n)
    #:
    #: Be sure to skip padding!
    ##

    masking = (y != 0)
    y = y[masking]
    y = y-1
    y_hat = y_hat[masking, :]
    average_loss = F.cross_entropy(y_hat, y)
    return average_loss

In [ ]:
def shift_left(tensor_list, pad_value=0.0):
    shifted_tensors = []
    for tensor in tensor_list:
        shifted_tensor = torch.zeros_like(tensor)
        shifted_tensor[:-1] = tensor[1:]
        shifted_tensor[-1] = pad_value
        shifted_tensors.append(shifted_tensor)

    return shifted_tensors

# Example usage:
input_ids = [torch.tensor([1, 2, 3, 4]), torch.tensor([5, 6, 7, 8])]
print("Input Ids:")
print(input_ids)

target_ids = shift_left(input_ids)
print("Shifted Left (Target Ids):")
print(target_ids)

Input Ids:
[tensor([1, 2, 3, 4]), tensor([5, 6, 7, 8])]
Shifted Left (Target Ids):
[tensor([2, 3, 4, 0]), tensor([6, 7, 8, 0])]


# Beam Search Generation

In [ ]:
import torch
import heapq

def tensor_to_string(tensor):
    chars = [chr(c) for c in tensor]
    return ''.join(chars)

def tensor_append_scalar(tensor, scalar):
    scalar_tensor = torch.tensor(scalar).view(1)  # Add a dimension to match the original tensor's dimensions
    scalar_tensor = scalar_tensor.to(device)

    # Append the scalar to the original tensor
    result = torch.cat((tensor, scalar_tensor), dim=0)
    return result


def generate_next_top_k(model, input_sequence, k):
    hidden = model.init_hidden(1).to(device, non_blocking=non_blocking)

    for char in input_sequence:
      output, hidden = model(torch.tensor([[char]]).to(device, non_blocking=non_blocking), hidden)
    logits = output
    probabilities = torch.softmax(logits, dim=-1)

    top_k_values, top_k_indices = torch.topk(probabilities, k)

    return [(tensor_append_scalar(input_sequence, idx.item() + 1), log_prob.item()) for idx, log_prob in zip(top_k_indices[0], top_k_values.log()[0])]

def beam_search(model, desired_length, starting_string, k=5):
    with ModelEvalMode(model), torch.no_grad():
      input_sequence = torch.tensor(str_to_np(starting_string).astype(np.int32) + 1, dtype=torch.long)
      input_sequence = input_sequence.to(device)

      log_prob = 0.0

      beam = [(input_sequence, log_prob)]

      while len(beam[0][0]) < desired_length:
          new_beam = []
          for seq, log_prob in beam:
              next_top_k = generate_next_top_k(model, seq, k)
              new_beam.extend([(new_seq, new_log_prob + log_prob) for new_seq, new_log_prob in next_top_k])

          beam = heapq.nlargest(k, new_beam, key=lambda x: x[1])

      return [tensor_to_string(seq - 1) for seq, _ in beam]

In [ ]:
def eval_gen(*args, display=999999, **kwargs):
    generated_texts = beam_search(
        *args, **kwargs,
    )

    for idx, text in enumerate(generated_texts):
        if idx >= display:
            break

        print(f"Generated text {idx + 1}: {text}")

# Train

In [ ]:
dt = dcs['train']
dt

Dataset({
    features: ['text', 'input'],
    num_rows: 13096
})

In [ ]:
#: Training Loop

import random
from torch.optim import lr_scheduler

if torch.cuda.is_available():
	device = 'cuda'
	non_blocking = True
elif True:
	device = 'cpu'
	non_blocking = False
else:
	device = 'mps'
	non_blocking = False

i = 0

epochs = 300
batch_size = 4096 * 2
learning_rate = 0.005
max_len = 40

m = Model().to(device=device, non_blocking=non_blocking)
m.train()

optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

counter = 0
for epoch in range(epochs):
	dt = dt.shuffle()
	for i in range(0, len(dt), batch_size):
		batch = dt[i:i+batch_size]
		inputs = batch['input']


		lens = [len(seq) for seq in inputs]
		current_max_len = max(lens)
		mean_len = statistics.mean(lens)

		clipped_inputs = []

		for index, input in enumerate(inputs):

			if lens[index] <= max_len:
				start_index = 0
				end_index = lens[index]
			else:
				start_index = random.randint(0, lens[index] - max_len)
				end_index = start_index + max_len

			clipped_input = torch.cat((input[start_index:end_index],
			                           torch.zeros(max(0, max_len- lens[index]))))
			clipped_inputs.append(clipped_input)

		inputs = clipped_inputs

		targets = shift_left(inputs)

		total_loss = 0
		hidden = m.init_hidden(len(inputs)).to(device, non_blocking=non_blocking)
		optimizer.zero_grad()
		for index in range(max_len):
			model_input = torch.tensor([x[index].item() for x in inputs]).to(device, non_blocking=non_blocking)
			model_target = torch.tensor([x[index].item() for x in targets]).to(device, non_blocking=non_blocking)

			model_output, hidden = m(model_input.int(), hidden)

			loss = loss_fn(model_target.long(), model_output)
			if torch.isnan(loss):
				continue
			total_loss += loss


		total_loss.backward()
		optimizer.step()

		l = total_loss / mean_len

		if counter % 1 == 0:
			l = l.item()
			print(f"loss: {l:>7f}  [{counter:>5d}, epoch={epoch}]")

		counter += 1

	if epoch % 60 == 0:
		eval_gen(display=2, model=m, desired_length=100, starting_string="Harry ", k=32)

	m = m.to(device, non_blocking=non_blocking)

loss: 1.162784  [    0, epoch=0]
loss: 1.003030  [    1, epoch=0]
Generated text 1: Harry                                                                                               
Generated text 2: Harry   a                                                                                           
loss: 0.741760  [    2, epoch=1]
loss: 0.784975  [    3, epoch=1]
loss: 0.803349  [    4, epoch=2]
loss: 0.778275  [    5, epoch=2]
loss: 0.726315  [    6, epoch=3]
loss: 0.683838  [    7, epoch=3]
loss: 0.632850  [    8, epoch=4]
loss: 0.650447  [    9, epoch=4]
loss: 0.658161  [   10, epoch=5]
loss: 0.612862  [   11, epoch=5]
loss: 0.611307  [   12, epoch=6]
loss: 0.605431  [   13, epoch=6]
loss: 0.601247  [   14, epoch=7]
loss: 0.585009  [   15, epoch=7]
loss: 0.573543  [   16, epoch=8]
loss: 0.572005  [   17, epoch=8]
loss: 0.557883  [   18, epoch=9]
loss: 0.566665  [   19, epoch=9]
loss: 0.552371  [   20, epoch=10]
loss: 0.540528  [   21, epoch=10]
loss: 0.537028  [   22, epoch=11]


In [ ]:
eval_gen(display=3, model=m, desired_length=20, starting_string="Harry ", k=32)

Generated text 1: Harry shook his head
Generated text 2: Harry couldn't help 
Generated text 3: Harry raised an eyeb


In [ ]:
eval_gen(display=4, model=m, desired_length=250, starting_string="Harry ", k=100)

Generated text 1: Harry rolled his eyes. 'Let's go!' shouted Daphne. 'What happened?' she asked suspiciously. 'What happened?' she asked suspiciously. 'What happened?' she asked suspiciously. 'What happened?' she asked suspiciously. 'What do you mean?' asked Hermione.
Generated text 2: Harry rolled his eyes. 'Let's go!' shouted Daphne. 'What happened?' she asked suspiciously. 'What happened?' she asked suspiciously. 'What happened?' she asked suspiciously. 'What happened?' she asked suspiciously. 'What happened?' she asked again. '
Generated text 3: Harry rolled his eyes. 'Let's go!' shouted Daphne. 'What happened?' she asked suspiciously. 'What happened?' she shouted. 'What happened?' she asked suspiciously. 'What happened?' she asked suspiciously. 'What happened?' she demanded suspiciously. 'I
Generated text 4: Harry rolled his eyes. 'Let's go!' shouted Daphne. 'What happened?' she asked suspiciously. 'What happened?' she asked suspiciously. 'What happened?' she asked suspiciously. '

In [ ]:
eval_gen(display=3, model=m, desired_length=250, starting_string="Arcturus ", k=100)

Generated text 1: Arcturus sighed, rubbing his eyes. 'Where are you talking about?' she demanded suspiciously. 'Why didn't you think about that, Hermione?' asked Hermione with a small voice of his head. 'What do you want to talk?' Hermione asked in a small voice from 
Generated text 2: Arcturus sighed, rubbing his eyes. 'Where are you talking about?' she demanded suspiciously. 'Why didn't you think about that, Hermione?' asked Hermione with a small voice of her head. 'What do you want to talk?' Hermione asked in a small voice from 
Generated text 3: Arcturus sighed, rubbing his eyes. 'Where are you talking about?' she demanded suspiciously. 'Why didn't you think about that, Hermione?' asked Hermione with a small voice of his head. 'What do you want to talk?' Harry asked in a small voice from the


In [ ]:
eval_gen(display=2, model=m, desired_length=150, starting_string="Draco ", k=100)

Generated text 1: Draco grinned. 'What are you talking about?' she demanded suspiciously. 'What are you talking about, Harry?' Hermione asked in a small voice from the 
Generated text 2: Draco grinned. 'What are you talking about?' she demanded suspiciously. 'What are you talking about?' Harry asked in a hushed voice. 'I can't tell you


In [ ]:
eval_gen(display=1, model=m, desired_length=50, starting_string="Harry looked at ", k=100)

Generated text 1: Harry looked at her, his eyes flickering towards t
